In [14]:
# !pip install transformers -q
# !pip install datasets -q
# !pip install evaluate -q

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"
import evaluate
import pandas as pd

df_train = pd.read_csv('/home/aflah20082/ML_Project/Data/PreprocessedData/english_train_preprocess.csv')
df_test = pd.read_csv('/home/aflah20082/ML_Project/Data/PreprocessedData/english_test_preprocess.csv')
df_val = pd.read_csv('/home/aflah20082/ML_Project/Data/PreprocessedData/english_dev_preprocess.csv')

In [16]:
label_replacement = {
    'Hope_speech': 0,
    'Non_hope_speech': 1,
    'not-English': 2,
}

df_train['label'] = df_train['label'].replace(label_replacement)
df_test['label'] = df_test['label'].replace(label_replacement)
df_val['label'] = df_val['label'].replace(label_replacement)

# Drop rows with label not-English
df_train = df_train[df_train['label'] != 2]
df_test = df_test[df_test['label'] != 2]
df_val = df_val[df_val['label'] != 2]

In [17]:
df_train = df_train[['preprocessed_text', 'label']]
df_test = df_test[['preprocessed_text', 'label']]
df_val = df_val[['preprocessed_text', 'label']]

In [18]:
df_train = df_train.rename(columns={'preprocessed_text': 'text'})
df_test = df_test.rename(columns={'preprocessed_text': 'text'})
df_val = df_val.rename(columns={'preprocessed_text': 'text'})

In [19]:
from datasets import Dataset, DatasetDict
tds = Dataset.from_pandas(df_train[['text', 'label']], preserve_index=False)
vds = Dataset.from_pandas(df_val[['text', 'label']], preserve_index=False)
testds = Dataset.from_pandas(df_test[['text', 'label']], preserve_index=False)


ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds
ds['test'] = testds

In [20]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# tokenizer.add_special_tokens(["<intent>", "</intent>"])

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [21]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

In [22]:
import numpy as np
metric1 = evaluate.load("accuracy")
metric2 = evaluate.load("precision")
metric3 = evaluate.load("recall")
metric4 = evaluate.load("f1")

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) 

training_args = TrainingArguments(output_dir=f"Model Dumps/bert_base_finetune", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy
1,0.285300,0.309273,0.913411
2,0.236100,0.268533,0.918339
3,0.240200,0.265282,0.917283


Saving model checkpoint to Model Dumps/bert_base_finetune/checkpoint-500
Configuration saved in Model Dumps/bert_base_finetune/checkpoint-500/config.json
Model weights saved in Model Dumps/bert_base_finetune/checkpoint-500/pytorch_model.bin
Saving model checkpoint to Model Dumps/bert_base_finetune/checkpoint-1000
Configuration saved in Model Dumps/bert_base_finetune/checkpoint-1000/config.json
Model weights saved in Model Dumps/bert_base_finetune/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to Model Dumps/bert_base_finetune/checkpoint-1500
Configuration saved in Model Dumps/bert_base_finetune/checkpoint-1500/config.json
Model weights saved in Model Dumps/bert_base_finetune/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to Model Dumps/bert_base_finetune/checkpoint-2000
Configuration saved in Model Dumps/bert_base_finetune/checkpoint-2000/config.json
Model weights saved in Model Dumps/bert_base_finetune/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to 

TrainOutput(global_step=8529, training_loss=0.2553329146303903, metrics={'train_runtime': 7826.1552, 'train_samples_per_second': 8.717, 'train_steps_per_second': 1.09, 'total_flos': 1.79494361966592e+16, 'train_loss': 0.2553329146303903, 'epoch': 3.0})

In [24]:
predictions = trainer.predict(tokenized_datasets["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
accuracy = metric1.compute(predictions=preds, references=predictions.label_ids)
precision = metric2.compute(predictions=preds, references=predictions.label_ids)
recall = metric3.compute(predictions=preds, references=predictions.label_ids)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 2841
  Batch size = 8


In [25]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)

Accuracy:  {'accuracy': 0.9172826469552974}
Precision:  {'precision': 0.949537750385208}
Recall:  {'recall': 0.959517321915142}


In [26]:
# Predictions on test set
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=-1)
accuracy = metric1.compute(predictions=preds, references=predictions.label_ids)
precision = metric2.compute(predictions=preds, references=predictions.label_ids)
recall = metric3.compute(predictions=preds, references=predictions.label_ids)
macro_f1 = metric4.compute(predictions=preds, references=predictions.label_ids, average='macro')
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("Macro F1: ", macro_f1)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 2843
  Batch size = 8


Accuracy:  {'accuracy': 0.9240239183960605}
Precision:  {'precision': 0.9551895825354271}
Recall:  {'recall': 0.9618202853837254}
Macro F1:  {'f1': 0.755180343240597}
